In [ ]:
# 1. Purge and install the "Clinical Stable" stack
!pip uninstall -y torch torchaudio torchcodec huggingface_hub speechbrain
!pip install torch==2.5.1 torchaudio==2.5.1 speechbrain==1.0.3 --quiet
!pip install "huggingface_hub<1.0" --quiet
!apt-get update && apt-get install -y ffmpeg --quiet

import os
import torch
import torchaudio


# 2. Monkeypatch: SpeechBrain looks for this in older TorchAudio,
# but we manually add it to prevent an import crash in the 2026 env.
if not hasattr(torchaudio, "list_audio_backends"):
    torchaudio.list_audio_backends = lambda: ["ffmpeg"]

print(f"✅ Environment Reset: Torch {torch.__version__} | Audio {torchaudio.__version__}")

# 3. FORCE RESTART: Ensures the fresh C++ binaries are linked correctly
# Note: The session will "crash" and restart - this is intentional!
os.kill(os.getpid(), 9)

Found existing installation: torch 2.10.0+cu128
Uninstalling torch-2.10.0+cu128:
  Successfully uninstalled torch-2.10.0+cu128
Found existing installation: torchaudio 2.10.0+cu128
Uninstalling torchaudio-2.10.0+cu128:
  Successfully uninstalled torchaudio-2.10.0+cu128
Found existing installation: torchcodec 0.10.0+cu128
Uninstalling torchcodec-0.10.0+cu128:
  Successfully uninstalled torchcodec-0.10.0+cu128
Found existing installation: huggingface_hub 1.4.1
Uninstalling huggingface_hub-1.4.1:
  Successfully uninstalled huggingface_hub-1.4.1
Found existing installation: speechbrain 1.0.3
Uninstalling speechbrain-1.0.3:
  Successfully uninstalled speechbrain-1.0.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━

In [2]:
import os
import torch
import torchaudio
import numpy as np
from google.colab import drive

# 1. Mount Drive if you haven't already
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. Path Setup (Ensure the folder name "Session_3" or "Session_003" is exact)
session_path = "/content/drive/My Drive/GEEKED/Audio_Files/Session_009"

doc_path = os.path.join(session_path, "doc_clean.wav")
pat_path = os.path.join(session_path, "pat_clean.wav")
import librosa
import numpy as np

# 1. Load both wav files into numpy arrays
# sr=16000 ensures they are both locked to the same sample rate
doctor_audio, sr1 = librosa.load(doc_path, sr=16000)
patient_audio, sr2 = librosa.load(pat_path, sr=16000)

# 2. Ensure they are the exact same length to avoid math errors
min_length = min(len(doctor_audio), len(patient_audio))
doctor_audio = doctor_audio[:min_length]
patient_audio = patient_audio[:min_length]


In [13]:
!pip install clearvoice

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.3/183.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 98.5 MB/s eta 0:00:00
  Created wheel for pyt

In [7]:
import torch
import torchaudio
import numpy as np
from clearvoice import ClearVoice
from IPython.display import Audio, display

# 1. Initialize the Engine
# (If you already ran this in a previous cell, you can skip this line)
mv_engine = ClearVoice(task='speech_separation', model_names=['MossFormer2_SS_16K'])

# 2. Path Setup
# We use the 'doc_clean.wav' which we know has both voices (crosstalk)
input_path = os.path.join(session_path, "doc_clean.wav")

print(f"🚀 MossFormer2 is unweaving identities from: {os.path.basename(input_path)}")

# 3. RUN THE ENGINE
# online_write=False returns the audio as a list of numpy arrays [array1, array2]
with torch.no_grad():
    separated_list = mv_engine(input_path=input_path, online_write=False)

# 4. EXTRACT AND DISPLAY
# Track 1 and Track 2 are now sitting directly in your RAM
# No more searching for hidden files!
track_0 = separated_list[0].squeeze()
track_1 = separated_list[1].squeeze()

print("\n✅ UNWEAVING COMPLETE")

print("\n--- Identity 1 (Usually Doctor) ---")
display(Audio(track_0, rate=16000))

print("\n--- Identity 2 (Usually Patient) ---")
display(Audio(track_1, rate=16000))

# --- THE TRUTH CHECK ---
# Check if they are actually different
similarity = np.corrcoef(track_0[:80000], track_1[:80000])[0, 1]
print(f"\nSeparation Power: {1 - abs(similarity):.4f} (1.0 = Perfect Separation)")

Output hidden; open in https://colab.research.google.com to view.